In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
DATA_PATH = os.environ.get("data_path")
sys.path.insert(0, os.getenv('lib_path'))
sys.path.insert(0, os.getenv('root_path'))

In [2]:
import pickle
import datetime
import pandas as pd
from lib.read_data import read_data
from lib.read_data_2015 import read_data_2015


In [3]:
from lib.filter_data import filter_data

In [4]:
from lib.boruta_selector import boruta_select_cols

In [5]:
%load_ext autoreload
%load_ext blackcellmagic
%autoreload 2

# Read data

In [ ]:
data_900 = read_data(
    DATA_PATH,
    merge_with_meds=True,
    merge_with_patient_genes=True,
)
data_900_df = data_900.input_df
data_2015_df = read_data_2015(DATA_PATH)

In [7]:
from lib.column_sets import prepare_column_sets

In [8]:
COL_SETS = prepare_column_sets(data_900, data_2015_df)

In [12]:
COL_SETS.__dict__.keys()

dict_keys(['DS1_PECH_COLS', 'MODULATED_COLS', 'DS1_RESULT_COLS', 'VARIOUS_COLS_900', 'BODY_COLS_900', 'HORMONE_COLS_900', 'HORMONE_NORMS_COLS_900', 'HORMONE_NORMS_VALID_IN_TIME_COLS_900', 'VALID_SUM_DOSES_COLS_900', 'VALID_SUM_DOSES_AND_DAYS_DIFF_COLS_900', 'CAUSE_COLS_900', 'VARIOUS_COLS_2015', 'HORMONE_COLS_2015', 'HORMONE_NORMS_COLS_2015', 'HORMONE_NORMS_VALID_IN_TIME_COLS_2015', 'FSH_NORM_CYCLED_COLS_2015', 'CAUSE_COLS_2015'])

# Try selected columns

In [98]:
test_columns_900 = [
    'ds1_pech_licz_14_pow',
    'ds1_pech_licz_18_pow',
    'ds1_pech_licz_11_pow',
    'ds1_pech_licz_10_pon',

    'bmi',
    'test_amh_r',
    'patient_age',
    'test_inhibina_b_r',
    'test_tsh_r',
    'test_e2_r',
    'test_fsh_r',
    'test_lh_r',
    'test_testosterone_r',
    'test_shbg_r',
    'test_dhea_s_r',
    "prev_proc-cumulus_denuded",
    "prev_proc-day_0_mii",
    
    'cause_idiopathic',
    'cause_tubal',
    'cause_pco',
    'cause_genetic_female',
    'cause_endocrine_thyroid',
    'cause_endocrine_other',
    'cause_immuno',
    'cause_rpl_rif',
    'cause_low_ovar_res',
    'cause_poi',
    'cause_endometriosis'
]

In [73]:
data_900_modelling = data_900_df.loc[data_900_df.test_amh_r.notna()&data_900_df.day_0_mii.notna()].copy()
data_900_modelling.cause_pco = data_900_modelling.cause_pco.cat.codes
data_900_modelling = filter_data(
    data_900_modelling, ~data_900_modelling["process_type"].isin(["DAWKJ", "BIOKJ", "DD", "DS"])
)
data_900_modelling = filter_data(
    data_900_modelling, ~data_900_modelling["lek_Gonadotropiny"].str.contains("Elonva")
)
data_900_modelling = filter_data(data_900_modelling, data_900_modelling["ds1_3_dawka_dzienna"] < 1250)
data_900_modelling = filter_data(data_900_modelling, data_900_modelling["test_amh_r"] < 15.0)
print(data_900_modelling.shape)
data_900_modelling.reset_index(inplace=True,drop=True)

(516, 1618)


In [12]:
from xgboost import XGBRegressor

In [15]:
from boruta import BorutaPy

In [22]:
import lightgbm as lgb

In [75]:
# sampling in proportion to y labels

rf = lgb.LGBMRegressor(n_jobs=-1, max_depth=5, n_estimators = 500, max_leaf_nodes= 5)

#rf = GradientBoostingClassifier(max_depth=3)


# To use Boruta that allows for missing data in X, use BorutaPyForLGB class

In [76]:
from sklearn.utils import check_random_state
import numpy as np

class BorutaPyForLGB(BorutaPy):
    def __init__(self, estimator, n_estimators=1000, perc=100, alpha=0.05,
                 two_step=True, max_iter=100, random_state=None, verbose=0):
        super().__init__(estimator, n_estimators, perc, alpha,
                         two_step, max_iter, random_state, verbose)
        self._is_lightgbm = 'lightgbm' in str(type(self.estimator))
        
    def _fit(self, X, y):
        # check input params
        # self._check_params(X, y)

        if not isinstance(X, np.ndarray):
            X = self._validate_pandas_input(X) 
        if not isinstance(y, np.ndarray):
            y = self._validate_pandas_input(y)

        self.random_state = check_random_state(self.random_state)
        # setup variables for Boruta
        n_sample, n_feat = X.shape
        _iter = 1
        # holds the decision about each feature:
        # 0  - default state = tentative in original code
        # 1  - accepted in original code
        # -1 - rejected in original code
        dec_reg = np.zeros(n_feat, dtype=np.int)
        # counts how many times a given feature was more important than
        # the best of the shadow features
        hit_reg = np.zeros(n_feat, dtype=np.int)
        # these record the history of the iterations
        imp_history = np.zeros(n_feat, dtype=np.float)
        sha_max_history = []

        # set n_estimators
        if self.n_estimators != 'auto':
            self.estimator.set_params(n_estimators=self.n_estimators)

        # main feature selection loop
        while np.any(dec_reg == 0) and _iter < self.max_iter:
            # find optimal number of trees and depth
            if self.n_estimators == 'auto':
                # number of features that aren't rejected
                not_rejected = np.where(dec_reg >= 0)[0].shape[0]
                n_tree = self._get_tree_num(not_rejected)
                self.estimator.set_params(n_estimators=n_tree)

            # make sure we start with a new tree in each iteration
            if self._is_lightgbm:
                self.estimator.set_params(random_state=self.random_state.randint(0, 10000))
            else:
                self.estimator.set_params(random_state=self.random_state)

            # add shadow attributes, shuffle them and train estimator, get imps
            cur_imp = self._add_shadows_get_imps(X, y, dec_reg)

            # get the threshold of shadow importances we will use for rejection
            imp_sha_max = np.percentile(cur_imp[1], self.perc)

            # record importance history
            sha_max_history.append(imp_sha_max)
            imp_history = np.vstack((imp_history, cur_imp[0]))

            # register which feature is more imp than the max of shadows
            hit_reg = self._assign_hits(hit_reg, cur_imp, imp_sha_max)

            # based on hit_reg we check if a feature is doing better than
            # expected by chance
            dec_reg = self._do_tests(dec_reg, hit_reg, _iter)

            # print out confirmed features
            if self.verbose > 0 and _iter < self.max_iter:
                self._print_results(dec_reg, _iter, 0)
            if _iter < self.max_iter:
                _iter += 1

        # we automatically apply R package's rough fix for tentative ones
        confirmed = np.where(dec_reg == 1)[0]
        tentative = np.where(dec_reg == 0)[0]
        # ignore the first row of zeros
        tentative_median = np.median(imp_history[1:, tentative], axis=0)
        # which tentative to keep
        tentative_confirmed = np.where(tentative_median
                                       > np.median(sha_max_history))[0]
        tentative = tentative[tentative_confirmed]

        # basic result variables
        self.n_features_ = confirmed.shape[0]
        self.support_ = np.zeros(n_feat, dtype=np.bool)
        self.support_[confirmed] = 1
        self.support_weak_ = np.zeros(n_feat, dtype=np.bool)
        self.support_weak_[tentative] = 1

        # ranking, confirmed variables are rank 1
        self.ranking_ = np.ones(n_feat, dtype=np.int)
        # tentative variables are rank 2
        self.ranking_[tentative] = 2
        # selected = confirmed and tentative
        selected = np.hstack((confirmed, tentative))
        # all rejected features are sorted by importance history
        not_selected = np.setdiff1d(np.arange(n_feat), selected)
        # large importance values should rank higher = lower ranks -> *(-1)
        imp_history_rejected = imp_history[1:, not_selected] * -1

        # update rank for not_selected features
        if not_selected.shape[0] > 0:
                # calculate ranks in each iteration, then median of ranks across feats
                iter_ranks = self._nanrankdata(imp_history_rejected, axis=1)
                rank_medians = np.nanmedian(iter_ranks, axis=0)
                ranks = self._nanrankdata(rank_medians, axis=0)

                # set smallest rank to 3 if there are tentative feats
                if tentative.shape[0] > 0:
                    ranks = ranks - np.min(ranks) + 3
                else:
                    # and 2 otherwise
                    ranks = ranks - np.min(ranks) + 2
                self.ranking_[not_selected] = ranks
        else:
            # all are selected, thus we set feature supports to True
            self.support_ = np.ones(n_feat, dtype=np.bool)

        self.importance_history_ = imp_history

        # notify user
        if self.verbose > 0:
            self._print_results(dec_reg, _iter, 1)
        return self

In [77]:

# define Boruta feature selection method
feat_selector = BorutaPyForLGB(rf, n_estimators='auto', verbose=0, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(data_900_modelling[test_columns_900].values, data_900_modelling['day_0_mii'].values.ravel())

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`

BorutaPyForLGB(estimator=LGBMRegressor(max_depth=5, max_leaf_nodes=5,
                                       n_estimators=40, random_state=1627),
               n_estimators='auto',
               random_state=RandomState(MT19937) at 0x7F438EDA5C40)

In [78]:
dict(zip(test_columns_900,feat_selector.ranking_))

{'ds1_pech_licz_14_pow': 15,
 'ds1_pech_licz_18_pow': 20,
 'ds1_pech_licz_11_pow': 16,
 'ds1_pech_licz_10_pon': 1,
 'bmi': 3,
 'test_amh_r': 1,
 'patient_age': 13,
 'test_inhibina_b_r': 4,
 'test_tsh_r': 2,
 'test_e2_r': 7,
 'test_fsh_r': 5,
 'test_lh_r': 10,
 'test_testosterone_r': 12,
 'test_shbg_r': 11,
 'test_dhea_s_r': 6,
 'prev_proc-cumulus_denuded': 9,
 'prev_proc-day_0_mii': 8,
 'cause_idiopathic': 25,
 'cause_tubal': 25,
 'cause_pco': 14,
 'cause_genetic_female': 25,
 'cause_endocrine_thyroid': 21,
 'cause_endocrine_other': 17,
 'cause_immuno': 21,
 'cause_rpl_rif': 19,
 'cause_low_ovar_res': 25,
 'cause_poi': 18,
 'cause_endometriosis': 25}

In [79]:
pd.Series(test_columns_900)[feat_selector.support_]

3    ds1_pech_licz_10_pon
5              test_amh_r
dtype: object

# Iterate backwards

In [80]:
from lib.data_series import prepare_data_serie

In [81]:
from lib.train import train_data_series

In [82]:
LGB_PARAMS_BASE = {
    "metric": ["rmse", "mae", "mse", "mape"],
    "boosting": "gbdt",
    "learning_rate": 0.05,
    "verbose": -1,
    "num_leaves": 16,
    "max_depth": 64,
    "max_bin": 63,
    "seed": 42,
    "num_threads": 10,
    "num_boost_round": 500,
    "early_stopping_round": 10
}

In [99]:
DATA_SERIES_900 = {}
DATA_SERIES_900['900_day_0_mii'] = prepare_data_serie(data_900_modelling, 'day_0_mii', 5)

Original records: 516
Filtered records: 516


In [84]:
pd.Series(test_columns_900)[feat_selector.support_]

3    ds1_pech_licz_10_pon
5              test_amh_r
dtype: object

In [85]:
selected_cols = pd.Series(test_columns_900)[feat_selector.support_].tolist()

In [100]:
Boruta_RESULTS = train_data_series(
    LGB_PARAMS_BASE,
    DATA_SERIES_900,
    selected_cols,
    model_suffixes_filter=['l2'])
Boruta_RESULTS.print_errors(
    [Boruta_RESULTS],
    print_suffixes=['l2'],
    print_avg=True,
)

Using default observation_weights
Using default observation_weights
Using default observation_weights
Using default observation_weights
Using default observation_weights
-
RMSE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   3.51    (0.00)    
MAE[ test_fold_avg][l2][900_day_0_mii]:        count:516    avg:5.92   2.60    (0.00)    
MAPE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   0.72    (0.00)    
LIKELIHOOD[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   3.30    (0.00)    
MEDIAN_ABSOLUTE_ERROR[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   1.96    (0.00)    
-----------------------------


In [88]:
order = dict(zip(test_columns_900,feat_selector.ranking_))

In [103]:
order = dict(sorted(order.items(), key=lambda item: item[1], reverse=True))

In [96]:
import shutup
shutup.please()

In [105]:
iter_columns = test_columns_900.copy()
for key, value in order.items():
    iter_columns.remove(key)
    print(f'Dropped {key}')
    Boruta_RESULTS = train_data_series(
        LGB_PARAMS_BASE,
        DATA_SERIES_900,
        iter_columns,
        model_suffixes_filter=['l2'])
    Boruta_RESULTS.print_errors(
        [Boruta_RESULTS],
        print_suffixes=['l2'],
        print_avg=True,
    )

Dropped cause_idiopathic
-
RMSE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   3.52    (0.00)    
MAE[ test_fold_avg][l2][900_day_0_mii]:        count:516    avg:5.92   2.59    (0.00)    
MAPE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   0.71    (0.00)    
LIKELIHOOD[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   3.30    (0.00)    
MEDIAN_ABSOLUTE_ERROR[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   2.01    (0.00)    
-----------------------------
Dropped cause_tubal
-
RMSE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   3.52    (0.00)    
MAE[ test_fold_avg][l2][900_day_0_mii]:        count:516    avg:5.92   2.59    (0.00)    
MAPE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   0.71    (0.00)    
LIKELIHOOD[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   3.30    (0.00)    
MEDIAN_ABSOLUTE_ERROR[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   2.01    (0.00)    

AMH and ds1_pech_licz_10 are by far the most important columns in the dataset. Check how the importance looks without them.

In [106]:
iter_columns = test_columns_900.copy()
iter_columns.remove('test_amh_r')
iter_columns.remove('ds1_pech_licz_10_pon')
for key, value in order.items():
    if key in iter_columns:
        iter_columns.remove(key)
        print(f'Dropped {key}')
        Boruta_RESULTS = train_data_series(
            LGB_PARAMS_BASE,
            DATA_SERIES_900,
            iter_columns,
            model_suffixes_filter=['l2'])
        Boruta_RESULTS.print_errors(
            [Boruta_RESULTS],
            print_suffixes=['l2'],
            print_avg=True,
        )

Dropped cause_idiopathic
-
RMSE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   4.31    (0.00)    
MAE[ test_fold_avg][l2][900_day_0_mii]:        count:516    avg:5.92   3.32    (0.00)    
MAPE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   0.97    (0.00)    
LIKELIHOOD[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   3.30    (0.00)    
MEDIAN_ABSOLUTE_ERROR[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   2.69    (0.00)    
-----------------------------
Dropped cause_tubal
-
RMSE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   4.31    (0.00)    
MAE[ test_fold_avg][l2][900_day_0_mii]:        count:516    avg:5.92   3.32    (0.00)    
MAPE[ test_fold_avg][l2][900_day_0_mii]:       count:516    avg:5.92   0.97    (0.00)    
LIKELIHOOD[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   3.30    (0.00)    
MEDIAN_ABSOLUTE_ERROR[ test_fold_avg][l2][900_day_0_mii]: count:516    avg:5.92   2.69    (0.00)    

There is a drastic increase of RMSE after prev-proc columns are removed. Keep in further analysis.